In [1]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.2 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, f1_score, accuracy_score, roc_auc_score
%matplotlib inline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [11]:
df_1 = pd.read_csv('binarized_data_1.csv')
df_2 = pd.read_csv('binarized_data_2.csv')
df_3 = pd.read_csv('binarized_data_3.csv')

# DataFrame 1


In [6]:
df_1 = df_1.drop('sample_id', axis=1)
y_feat = 'is_setosa'
df_train, df_test = train_test_split(df_1, train_size=0.7, random_state=0)
X_train, y_train = df_train.drop(y_feat, axis=1), df_train[y_feat]
X_test, y_test = df_test.drop(y_feat, axis=1), df_test[y_feat]
X_train

,"sepal_length(4.3, 5.3)","sepal_length(5.3, 6.3)","sepal_length(6.3, 7.3)","sepal_length(7.3, 8.3)","sepal_width(2.0, 3.25)","sepal_width(3.25, 4.5)","petal_length(1.0, 3.0)","petal_length(3.0, 5.0)","petal_length(5.0, 7.0)","petal_width(0.1, 1.1)","petal_width(1.1, 2.1)","petal_width(2.1, 3.1)"
60,True,False,False,False,True,False,False,True,False,True,False,False
116,False,False,True,False,True,False,False,False,True,False,True,False
144,False,False,True,False,False,True,False,False,True,False,False,True
119,False,True,False,False,True,False,False,False,True,False,True,False
108,False,False,True,False,True,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
9,True,False,False,False,True,False,True,False,False,True,False,False
103,False,False,True,False,True,False,False,False,True,False,True,False
67,False,True,False,False,True,False,False,True,False,True,False,False
117,False,False,False,True,False,True,False,False,True,False,False,True


In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

X = X_test
y = y_test

results = {
    'Model': [],
    'Accuracy': [],
    'F1 Score': [],
    'ROC-AUC': []
}

models = {
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(),
    'KNN': KNeighborsClassifier(),
    'NaiveBayes': GaussianNB(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
}

params = {
    'DecisionTree': {'max_depth': [None, 5, 10]},
    'RandomForest': {'n_estimators': [10, 50], 'max_depth': [None, 5]},
    'KNN': {'n_neighbors': [3, 5, 7]},
    'NaiveBayes': {},
    'LogisticRegression': {'C': [0.1, 1, 10]},
}

for name, model in models.items():
    clf = GridSearchCV(model, params[name], cv=5, scoring=make_scorer(f1_score, average='macro'))
    clf.fit(X, y)

    best_model = clf.best_estimator_


    acc_scores = cross_val_score(best_model, X, y, cv=5, scoring='accuracy')
    f1_scores = cross_val_score(best_model, X, y, cv=5, scoring=make_scorer(f1_score, average='macro'))

    predicted_probs = cross_val_predict(best_model, X, y, cv=5, method='predict_proba')[:, 1]
    roc_auc = roc_auc_score(y, predicted_probs)

    results['Model'].append(name)
    results['Accuracy'].append(round(acc_scores.mean(),3))
    results['F1 Score'].append(round(f1_scores.mean(),3))
    results['ROC-AUC'].append(round(roc_auc, 3))

df_results = pd.DataFrame(results)
print(df_results)

                Model  Accuracy  F1 Score  ROC-AUC
0        DecisionTree     1.000     1.000      1.0
1        RandomForest     1.000     0.977      1.0
2                 KNN     0.978     0.977      1.0
3          NaiveBayes     1.000     1.000      1.0
4  LogisticRegression     1.000     1.000      1.0


In [11]:
df_2 = df_2.drop('sample_id', axis=1)
y_feat = 'is_versicolor'
df_train, df_test = train_test_split(df_2, train_size=0.7, random_state=0)
X_train, y_train = df_train.drop(y_feat, axis=1), df_train[y_feat]
X_test, y_test = df_test.drop(y_feat, axis=1), df_test[y_feat]
X_train

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

X = X_test
y = y_test

results = {
    'Model': [],
    'Accuracy': [],
    'F1 Score': [],
    'ROC-AUC': []
}

models = {
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(),
    'KNN': KNeighborsClassifier(),
    'NaiveBayes': GaussianNB(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
}

params = {
    'DecisionTree': {'max_depth': [None, 5, 10]},
    'RandomForest': {'n_estimators': [10, 50], 'max_depth': [None, 5]},
    'KNN': {'n_neighbors': [3, 5, 7]},
    'NaiveBayes': {},
    'LogisticRegression': {'C': [0.1, 1, 10]},
}

for name, model in models.items():
    clf = GridSearchCV(model, params[name], cv=5, scoring=make_scorer(f1_score, average='macro'))
    clf.fit(X, y)

    best_model = clf.best_estimator_


    acc_scores = cross_val_score(best_model, X, y, cv=5, scoring='accuracy')
    f1_scores = cross_val_score(best_model, X, y, cv=5, scoring=make_scorer(f1_score, average='macro'))

    predicted_probs = cross_val_predict(best_model, X, y, cv=5, method='predict_proba')[:, 1]
    roc_auc = roc_auc_score(y, predicted_probs)

    results['Model'].append(name)
    results['Accuracy'].append(round(acc_scores.mean(),3))
    results['F1 Score'].append(round(f1_scores.mean(),3))
    results['ROC-AUC'].append(round(roc_auc, 3))

df_results = pd.DataFrame(results)
print(df_results)

                Model  Accuracy  F1 Score  ROC-AUC
0        DecisionTree     0.889     0.887    0.893
1        RandomForest     0.889     0.864    0.959
2                 KNN     0.867     0.864    0.908
3          NaiveBayes     0.844     0.841    0.861
4  LogisticRegression     0.889     0.886    0.905


In [12]:
df_3 = df_3.drop('sample_id', axis=1)
y_feat = 'is_virginica'
df_train, df_test = train_test_split(df_3, train_size=0.7, random_state=0)
X_train, y_train = df_train.drop(y_feat, axis=1), df_train[y_feat]
X_test, y_test = df_test.drop(y_feat, axis=1), df_test[y_feat]
X_train

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

X = X_test
y = y_test

results = {
    'Model': [],
    'Accuracy': [],
    'F1 Score': [],
    'ROC-AUC': []
}

models = {
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(),
    'KNN': KNeighborsClassifier(),
    'NaiveBayes': GaussianNB(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
}

params = {
    'DecisionTree': {'max_depth': [None, 5, 10]},
    'RandomForest': {'n_estimators': [10, 50], 'max_depth': [None, 5]},
    'KNN': {'n_neighbors': [3, 5, 7]},
    'NaiveBayes': {},
    'LogisticRegression': {'C': [0.1, 1, 10]},
}

for name, model in models.items():
    clf = GridSearchCV(model, params[name], cv=5, scoring=make_scorer(f1_score, average='macro'))
    clf.fit(X, y)

    best_model = clf.best_estimator_


    acc_scores = cross_val_score(best_model, X, y, cv=5, scoring='accuracy')
    f1_scores = cross_val_score(best_model, X, y, cv=5, scoring=make_scorer(f1_score, average='macro'))

    predicted_probs = cross_val_predict(best_model, X, y, cv=5, method='predict_proba')[:, 1]
    roc_auc = roc_auc_score(y, predicted_probs)

    results['Model'].append(name)
    results['Accuracy'].append(round(acc_scores.mean(),3))
    results['F1 Score'].append(round(f1_scores.mean(),3))
    results['ROC-AUC'].append(round(roc_auc, 3))

df_results = pd.DataFrame(results)
print(df_results)

                Model  Accuracy  F1 Score  ROC-AUC
0        DecisionTree     0.844     0.643    0.864
1        RandomForest     0.889     0.780    0.909
2                 KNN     0.889     0.780    0.866
3          NaiveBayes     0.644     0.625    0.821
4  LogisticRegression     0.911     0.852    0.893


# DataFrame 2

,hypertension,heart_disease,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,...,smoking_status_smokes,age(0-30),age(30-60),age(60-90),avg_glucose_level(55-155),avg_glucose_level(155-255),avg_glucose_level(255-355),bmi(10-50),bmi(50-90),bmi(90-130)
1921,False,False,False,True,False,False,True,False,False,True,...,True,False,True,False,True,False,False,True,False,False
4074,False,False,True,False,False,False,True,True,False,False,...,False,False,False,True,False,True,False,True,False,False
2311,False,False,True,False,False,True,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
3080,False,False,True,False,False,False,True,False,False,True,...,False,False,False,True,True,False,False,True,False,False
1620,False,False,True,False,False,False,True,True,False,False,...,False,False,False,True,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4931,True,False,True,False,False,False,True,True,False,False,...,True,False,True,False,True,False,False,True,False,False
3264,False,False,True,False,False,False,True,False,False,True,...,False,False,False,True,True,False,False,True,False,False
1653,False,False,False,True,False,False,True,False,False,True,...,True,False,True,False,True,False,False,True,False,False
2607,False,False,False,True,False,True,False,False,False,True,...,False,True,False,False,True,False,False,True,False,False


In [12]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict
from sklearn.metrics import f1_score, roc_auc_score, make_scorer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

# Ваш предыдущий код
df_2 = df_2.drop('sample_id', axis=1)
y_feat = 'stroke'
df_train, df_test = train_test_split(df_2, train_size=0.7, random_state=0)
X_train, y_train = df_train.drop(y_feat, axis=1), df_train[y_feat]
X_test, y_test = df_test.drop(y_feat, axis=1), df_test[y_feat]
# Преобразование булевых значений в числовой формат
X_train = X_train.astype(int)
X_test = X_test.astype(int)
y_train = y_train.astype(int)
y_test = y_test.astype(int)
# Применение SMOTE к обучающему набору
sm = SMOTE(random_state=0)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

# Определение моделей и параметров
models = {
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(),
    'KNN': KNeighborsClassifier(),
    'NaiveBayes': GaussianNB(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
}

params = {
    'DecisionTree': {'max_depth': [None, 5, 10]},
    'RandomForest': {'n_estimators': [10, 50], 'max_depth': [None, 5]},
    'KNN': {'n_neighbors': [3, 5, 7]},
    'NaiveBayes': {},
    'LogisticRegression': {'C': [0.1, 1, 10]},
}

# Обучение моделей и валидация
results = {
    'Model': [],
    'Accuracy': [],
    'F1 Score': [],
    'ROC-AUC': []
}

for name, model in models.items():
    clf = GridSearchCV(model, params[name], cv=5, scoring=make_scorer(f1_score, average='macro'))
    clf.fit(X_train_sm, y_train_sm)  # Обучение на сбалансированных данных

    # Применение модели к тестовой выборке
    acc_scores = cross_val_score(clf.best_estimator_, X_test, y_test, cv=5, scoring='accuracy')
    f1_scores = cross_val_score(clf.best_estimator_, X_test, y_test, cv=5, scoring=make_scorer(f1_score, average='macro'))
    predicted_probs = cross_val_predict(clf.best_estimator_, X_test, y_test, cv=5, method='predict_proba')[:, 1]
    roc_auc = roc_auc_score(y_test, predicted_probs)

    results['Model'].append(name)
    results['Accuracy'].append(round(acc_scores.mean(),3))
    results['F1 Score'].append(round(f1_scores.mean(),3))
    results['ROC-AUC'].append(round(roc_auc, 3))

df_results = pd.DataFrame(results)
print(df_results)

                Model  Accuracy  F1 Score  ROC-AUC
0        DecisionTree     0.945     0.537    0.546
1        RandomForest     0.959     0.489    0.690
2                 KNN     0.960     0.505    0.561
3          NaiveBayes     0.259     0.229    0.710
4  LogisticRegression     0.962     0.507    0.768


# DataFrame 3

In [3]:
df_3 = pd.read_csv('binarized_data_3 (2).csv')
df_3 = df_3.drop('sample_id', axis=1)
y_feat = 'satisfaction'
df_train, df_test = train_test_split(df_3, train_size=0.7, random_state=0)
X_train, y_train = df_train.drop(y_feat, axis=1), df_train[y_feat]
X_test, y_test = df_test.drop(y_feat, axis=1), df_test[y_feat]
X_train

,Inflight wifi service_high,Departure/Arrival time convenient_high,Ease of Online booking_high,Gate location_high,Food and drink_high,Online boarding_high,Seat comfort_high,Inflight entertainment_high,On-board service_high,Leg room service_high,...,Gender_Female,Gender_Male,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Departure_Is_Delay,Arrival_Is_Delay
70230,False,False,False,False,False,False,False,False,False,False,...,True,False,True,False,True,False,False,True,False,False
87097,False,False,False,False,False,True,True,True,True,True,...,True,False,True,False,True,False,True,False,True,False
74136,False,False,False,False,True,True,True,True,True,True,...,False,True,True,False,True,False,True,False,False,False
58480,False,False,False,False,False,False,False,False,True,False,...,False,True,False,True,True,False,False,True,False,False
25055,False,False,False,True,True,False,False,True,False,False,...,True,False,False,True,True,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21243,True,True,True,False,False,True,False,False,True,False,...,True,False,True,False,False,True,False,True,False,False
45891,False,False,False,False,True,True,True,True,False,False,...,False,True,True,False,True,False,True,False,True,True
42613,True,False,True,True,False,True,False,False,False,False,...,True,False,False,True,True,False,True,False,False,False
43567,True,False,True,False,False,True,False,False,False,True,...,False,True,False,True,True,False,True,False,False,False


In [4]:
X = X_test
y = y_test

results = {
    'Model': [],
    'Accuracy': [],
    'F1 Score': [],
    'ROC-AUC': []
}

models = {
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(),
    'KNN': KNeighborsClassifier(),
    'NaiveBayes': GaussianNB(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
}

params = {
    'DecisionTree': {'max_depth': [None, 5, 10]},
    'RandomForest': {'n_estimators': [10, 50], 'max_depth': [None, 5]},
    'KNN': {'n_neighbors': [3, 5, 7]},
    'NaiveBayes': {},
    'LogisticRegression': {'C': [0.1, 1, 10]},
}

for name, model in models.items():
    clf = GridSearchCV(model, params[name], cv=5, scoring=make_scorer(f1_score, average='macro'))
    clf.fit(X, y)

    best_model = clf.best_estimator_


    acc_scores = cross_val_score(best_model, X, y, cv=5, scoring='accuracy')
    f1_scores = cross_val_score(best_model, X, y, cv=5, scoring=make_scorer(f1_score, average='macro'))


    predicted_probs = cross_val_predict(best_model, X, y, cv=5, method='predict_proba')[:, 1]
    roc_auc = roc_auc_score(y, predicted_probs)

    results['Model'].append(name)
    results['Accuracy'].append(round(acc_scores.mean(),3))
    results['F1 Score'].append(round(f1_scores.mean(),3))
    results['ROC-AUC'].append(round(roc_auc, 3))

df_results = pd.DataFrame(results)
print(df_results)

                Model  Accuracy  F1 Score  ROC-AUC
0        DecisionTree     0.897     0.896    0.942
1        RandomForest     0.894     0.891    0.951
2                 KNN     0.893     0.890    0.942
3          NaiveBayes     0.858     0.856    0.915
4  LogisticRegression     0.889     0.887    0.943
